- The purpose of this note is to show you how to use selenium in kaggle notebook.
- Please, do upvote if you find it uselful or intersteing.

# Install

In [ ]:
# install google chrome
!wget https://dl.google.com/linux/linux_signing_key.pub
!sudo apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!sudo apt-get -y update
!sudo apt-get install -y google-chrome-stable

--2023-07-25 06:36:12--  https://dl.google.com/linux/linux_signing_key.pub
Resolving dl.google.com (dl.google.com)... 142.251.18.93, 142.251.18.190, 142.251.18.91, ...
Connecting to dl.google.com (dl.google.com)|142.251.18.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14605 (14K) [application/octet-stream]
Saving to: ‘linux_signing_key.pub.1’

linux_signing_key.p 100%[===================>]  14.26K  --.-KB/s    in 0s      

2023-07-25 06:36:12 (62.7 MB/s) - ‘linux_signing_key.pub.1’ saved [14605/14605]

OK
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://dl.google.com/linux/chrome/deb stable InRelease                  
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease      
Get:5 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5002 B]
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease    
Get:7 http

In [ ]:
# install chromedriver
# !apt-get install -y qq unzip
!wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/

--2023-07-25 06:36:23--  http://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 173.194.69.128, 173.194.79.128, 108.177.96.128, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|173.194.69.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7407250 (7.1M) [application/zip]
Saving to: ‘/tmp/chromedriver.zip’

/tmp/chromedriver.z 100%[===================>]   7.06M  8.99MB/s    in 0.8s    

2023-07-25 06:36:23 (8.99 MB/s) - ‘/tmp/chromedriver.zip’ saved [7407250/7407250]

Archive:  /tmp/chromedriver.zip
replace /usr/local/bin/chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [ ]:
# install selenium
!sudo apt install -y python3-selenium
!pip install selenium==3.141.0 > /dev/null

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-selenium is already the newest version (4.0.0~a1+dfsg1-1.1).
0 upgraded, 0 newly installed, 0 to remove and 248 not upgraded.
W: Target Packages (main/binary-amd64/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google-chrome.list:4
W: Target Packages (main/binary-all/Packages) is configured multiple times in /etc/apt/sources.list.d/google-chrome.list:3 and /etc/apt/sources.list.d/google-chrome.list:4


In [ ]:
# To check Google Chrome's version
!google-chrome --version

Google Chrome 115.0.5790.102 


In [ ]:
# To check Chrome Driver's version
!chromedriver -v

ChromeDriver 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})


# Let's get started!

In [ ]:
# import libraries
import time
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
url = 'https://www.kaggle.com/competitions'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
from bs4 import BeautifulSoup
from tqdm import tqdm
# Define the URL of the website
url = 'https://rentola.com/for-rent?utf8=%E2%9C%93&location=&location=london'


driver.get(url)


elem = driver.find_element(by =By.CLASS_NAME, value = 'coi-banner__accept')
try :

    elem.click()
except:
    pass


for i in tqdm(range(33)):
    time.sleep(1)
    load_more = driver.find_element(by =By.ID, value = 'load-more')
    if load_more:
        driver.execute_script("arguments[0].click();", load_more)


100%|██████████| 33/33 [09:26<00:00, 17.15s/it]


In [ ]:
html_content = driver.page_source

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(html_content, 'html.parser')

In [ ]:
listings = []
for k in soup.find_all('a'):
    try:
        if k['href'].startswith('/listings') and  k['href'] !='/listings/new':
            listings.append(k['href'])
    except:
        pass
print(len(listings))
listings = [listings[i] for i in range(0, len(listings),2)]

1782


In [ ]:
import requests
all_specs = []

for propert in tqdm(listings):
    xx = requests.get(f'https://rentola.com{propert}')


    html_content = xx.text

    # Parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')
    spec_dict = {}

    spec_dict['title'] = soup.find('h1').text.strip('\n')
    spec_dict['address'] = soup.find('h2').text
    spec_dict['link'] = f'https://rentola.com{propert}'

    for spec in soup.find_all(class_ ='circle'):
        feature = spec.find(class_ = 'about-label').text
        value = spec.find(class_ = 'data').text
        spec_dict[feature] = value
    try:
        spec_dict['description'] = soup.find(itemprop = 'description').text
    except (AttributeError):
        pass
    try:
        spec_dict['latitude'] = soup.find('div', id = 'map')['data-lat']
        spec_dict['longitude'] = soup.find('div', id = 'map')['data-lng']
    except:
        pass
    all_specs.append(spec_dict)


 78%|███████▊  | 697/891 [16:11<04:59,  1.55s/it] 

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(all_specs, orient='columns')
df.to_csv('Rentola-1.csv',index = False)
df

In [ ]:
# Send a GET request to the website and retrieve the HTML content
#response = scraper.get(url)


# Extract the desired information from the parsed HTML
# You'll need to inspect the HTML structure of the website to find the appropriate elements and attributes to extract the data you want
# For example, if the information you want is contained in <div> elements with class 'property-item', you can use the following code:
property_items = soup.find_all('div', class_='property')
print(property_items)

# Iterate over the property items and extract the desired data
for item in property_items:
    # Extract the URL of the property
    property_url = item.find('a')['href']

    # Extract the number of bedrooms
    bedrooms = item.find('span', class_='bedrooms').text

    # Extract the number of bathrooms
    bathrooms = item.find('span', class_='bathrooms').text

    # Extract the apartment type
    apartment_type = item.find('span', class_='apartment-type').text

    # Extract the price
    price = item.find('span', class_='price').text

    # Extract the location
    location = item.find('span', class_='location').text

    # Print the extracted data
    print('URL:', property_url)
    print('Bedrooms:', bedrooms)
    print('Bathrooms:', bathrooms)
    print('Apartment Type:', apartment_type)
    print('Price:', price)
    print('Location:', location)
    print('---')


In [ ]:
elem = driver.find_element(by =By.CLASS_NAME, 'coi-banner__accept')
elem.click()

In [ ]:
# get competition "All competition" button
elem = driver.find_element(by=By.XPATH, value='//*[@id="site-content"]/div[4]/div[4]/div/div[2]/div/button[1]')
# button click
elem.click()
# javascript is running, so wait until to get the information.
time.sleep(5)

In [ ]:
# take a picture
driver.save_screenshot('result.png')
img = Image.open('result.png')
img